In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('resources/secondset.csv',
                 index_col='transactionId',
                 usecols=[0, 1, 2, 3, 4])

In [3]:
df

,customerId,accountId,transactionType,amount
transactionId,,,,
1,14,22,withdrawal,663.59
2,14,21,withdrawal,955.41
3,10,20,withdrawal,238.24
4,11,24,withdrawal,542.67
5,14,25,withdrawal,394.00
6,14,26,deposit,480.25
7,12,22,withdrawal,733.03
8,12,26,withdrawal,666.25
9,12,23,withdrawal,374.90


In [4]:
balances = df.set_index(['customerId', 'accountId']).sort_index()

In [5]:
balances = balances.drop(['transactionType', 'amount'], axis=1)

In [7]:
balances['balance']=0

In [8]:
balances

balance
customerId accountId         
10         20               0
           22               0
           25               0
11         21               0
           24               0
12         20               0
           21               0
           22               0
           22               0
           23               0
           24               0
           26               0
13         24               0
           25               0
14         21               0
           21               0
           22               0
           24               0
           25               0
           26               0